In [2]:
### import libraries

import wradlib as wrl
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import datetime as dt
from osgeo import osr
import tarfile
import os
import math

In [4]:
def project_radar_data(filename, EPSG, AntennaElev, var):
    """
      Objective: Project the 3D radar data to a coordinate reference system (CRS) specified by an EPSG.

      Inputs:
          - filename: Name of the hdf5 file to be projected
          - EPSG: EPSG code of the coordinate reference system to which the dta will be projected
          - AntennaElev: Elevation of the antenna above sea level
          - var: Name of the variable to be projected (e.g. Z, Zv, CCOR, CPA, etc)

      Outputs:
          - data: Array with the values of the variable specified
          - xyz: 3D coordinates of each value
          - rad_dict: dictionary with data from the radar (e.g. name, num_scans, sitecoords, elevation angles)

    """
    # Read radar data
    raw = wrl.io.read_opera_hdf5(filename)

    # Name of the antenna
    name = str(raw['radar1']['radar_name']).split("'")[1]

    #Time of measurement
    time = filename.split('_')[-1].split('.')[0]
    time = datetime.strptime(time, '%Y%m%d%H%M')

    #print('Projecting data for', name,'/', time)

    # Get number of scans
    num_scans = raw['overview']['number_scan_groups'][0]

    # this is the radar position tuple (longitude, latitude, altitude)
    sitecoords = (raw['radar1']['radar_location'][0], raw['radar1']['radar_location'][1])

    # define the cartesian reference system using the EPSG
    proj = osr.SpatialReference()
    proj.ImportFromEPSG(EPSG)


    # Empty arrays to hold Cartesian coordinates and data


    global xyz
    global data

    xyz, data = np.array([]).reshape((-1, 3)), np.array([])

    elevs = []

    for i in range(14):

        # get the scan metadata, data and calibration for each elevation
        meta = raw['scan'+str(i+1)]
        what = raw['scan'+str(i+1)+'/scan_'+var+'_data']
        calib = raw['scan'+str(i+1)+'/calibration']

        naray = what.shape[0]
        nbins = what.shape[1]

        # define variable with elevation angle and append it to elevs list
        el = meta['scan_elevation'][0]
        elevs.append(el)

        # define array with azimuth angles
        az = np.arange(0.0, 360.0, 360.0 / naray).reshape(360,1)


        bin_range = (299792458./(2.*meta['scan_high_PRF'][0]))/nbins

        # maximum range for the KNMI radar dataset is supposed to be 200 km.
        r = np.linspace(0,nbins*bin_range, nbins)
        # maximum range for the KNMI radar dataset is assumed to be 200 km.
        # r = np.linspace(0,200000, nbins)

        # Calibrate data (NEED TO CHECK THIS)
        multi = float(str(calib['calibration_'+var+'_formulas']).split('=')[1].split('*')[0])
        sum_ = float(str(calib['calibration_'+var+'_formulas']).split('+')[1].split("'")[0])

        data_ = sum_ + multi * what

        site = (sitecoords[0], sitecoords[1], AntennaElev) # Check if elevation = 50m

        # Get volumetric coordinates
        xyz_ = wrl.vpr.volcoords_from_polar(site, el, az, ranges = r, proj = proj)


        xyz, data = np.vstack((xyz, xyz_)), np.append(data, data_.ravel())

    # Radar characteristics
    

    rad_dict = {'name' : name,
                'time': time,
                'num_scans' : num_scans,
                'sitecoords' : sitecoords,
                'elev_ang' : elevs,
                'proj':proj}

    return data, xyz, rad_dict



In [5]:
def interpolate_radar_data(data, coords, proj, sitecoords, maxrange, minelev, maxelev, maxalt, horiz_res, vert_res):
    """
      Objective: Function to interpolate the data in a 3D volume grid

      Inputs:
          - data: Array with the values of a specified variable
          - coords: Array with the xyz coordinates of the specified variable
          - proj: CRS to which the data will be projected
          - sitecoords: Location of the radar antenna
          - Remaining are related to the resolution desired

      Outputs:
          - vol: Masked array with the values of the specified variable
          - xyz: list with x, y and z the coordinates
    """
    # Create 3D grid
    trgxyz, trgshape = wrl.vpr.make_3d_grid(sitecoords, proj, maxrange, maxalt, horiz_res, vert_res)

    x = trgxyz[:, 0].reshape(trgshape)[0, 0, :]
    y = trgxyz[:, 1].reshape(trgshape)[0, :, 0]
    z = trgxyz[:, 2].reshape(trgshape)[:, 0, 0]

    xyz = [x, y, z]

    # interpolate to Cartesian 3-D volume grid
    gridder = wrl.vpr.CAPPI(coords, trgxyz, trgshape, maxrange, minelev, maxelev)
    vol = np.ma.masked_invalid(gridder(data).reshape(trgshape))

    return vol, xyz

In [6]:
def rasterize_agg(vol, coords, res_params, EPSG,agg_fun = np.max):
    """
      Objective: Function to rasterize the volume using an aggregating function

      Inputs:
          - vol: Masked array with the values of the specified variable.
          - coords: list with the x, y, z coordinates.
          - res_params: dictionary with the resolution parameters.
          - EPSG: Code of CRS to which the data will be projected.
          - output_tif: Filename of the tif file.

      Outputs:
          - vol: sliced volume for the spcified Elev_ASL
          - XY: coordinates for the specified Elev_ASL

    """


    x, y, z = coords[0], coords[1], coords[2]

    xres = res_params['horiz_res']
    yres = res_params['horiz_res']

    vol = agg_fun(vol, axis = 0)
    
    side = round((len(vol)-round(math.sqrt(len(vol)**2/2)))/2)
    vol_ = vol[side:len(vol)-side,side:len(vol)-side]

    return vol_

In [8]:
import shutil

# Define the parameters
EPSG = 32632
var = "Z"
AntennaElev = 50
res_params = dict(maxrange = 200000.0,
                  maxalt = 15000.0,
                  horiz_res = 1980.0, # 1980(144),1780(160), 1480(192)
                  vert_res = 100.0)

# Iterate over the downloaded dates
dates = [20230111, 20230115, 20230424, 20230510, 20230705, 20230731, 20230801, 20230803, 20230806, 20230922, 20231013, 20231019, 20231021, 20231031, 20231103, 20231109, 20231119, 20231127, 20231210]

for date in dates:
    '''
    Function to pre-process the radar data for all the given dates
    '''
    
    # Convert date to string
    date= str(date)
    dateObject = dt.datetime.strptime(date, '%Y%m%d')
    # Find the date file
    filename = f"RAD62_OPER_O___TARVOL__L2__{date}T000000_{(dateObject + dt.timedelta(days=1)).strftime('%Y%m%d')}T000000_0001.tar"
    
    # Extract the file
    if not os.path.exists("Weather_3D"):
        os.mkdir("Weather_3D")
    
    fname = os.path.join("Weather_3D", date)
    with tarfile.open(filename, "r") as tf:
        if os.path.isdir(f"{fname}"):
            shutil.rmtree(f"{fname}")
            
        os.mkdir(f"Weather_3D/{date}")
        tf.extractall(path=f"Weather_3D/{date}",filter='data')


    # initialise
    if not os.path.exists("Sequences"):
        os.mkdir("Sequences")
             
    Video = []
    iteration = 0
    hour = 0

    for file in os.listdir(fname):
        '''
        Iterate over the extracted files
        '''
        file= os.path.join(fname, file)
        iteration += 1

        # Function for vertical aggregation (e.g. np.max, np.quantile, np.mean, etc.)
        agg_fun = np.max

        # STEP 1: Project the radar volume
        data, xyz, rad_d = project_radar_data(file, EPSG, AntennaElev, var)

        #remove negative values and normalise data
        data[data<0] = 0
        data = data/70

        # Set elevation angles in resolution parameters
        res_params['minelev'] = np.min(rad_d['elev_ang'])
        res_params['maxelev'] = np.max(rad_d['elev_ang'])

        # STEP 2: interpolate to Cartesian 3-D volume grid
        v, co = interpolate_radar_data(data, xyz, rad_d['proj'], rad_d['sitecoords'], **res_params)

        # STEP 3: Rasterize the 3D grid with vertical aggregation
        vol = rasterize_agg(v, co, res_params, EPSG, agg_fun)

        # Remove the NaN values and save the 2D image
        Image = np.ma.getdata(vol)
        Image[Image==1e+20] = 0
        Video.append([Image])
        np.save(f"Sequences/{date}.npy",np.array(Video))
        
    # Remove the extracted file to save space
    if os.path.isdir(f"{fname}"):
        shutil.rmtree(f"{fname}")
        



c:\Users\Aleja\anaconda3\envs\geospatial\Lib\site-packages\wradlib\vpr.py:103: DeprecationWarning: ``gridshape`` is not used in <class 'wradlib.vpr.CAPPI'>. It will be removed in wradlib version 2.0.
  warnings.warn(


appended image Weather_3D\20230115\RAD_NL62_VOL_NA_202301150000.h5


c:\Users\Aleja\anaconda3\envs\geospatial\Lib\site-packages\wradlib\vpr.py:103: DeprecationWarning: ``gridshape`` is not used in <class 'wradlib.vpr.CAPPI'>. It will be removed in wradlib version 2.0.
  warnings.warn(


appended image Weather_3D\20230115\RAD_NL62_VOL_NA_202301150005.h5


c:\Users\Aleja\anaconda3\envs\geospatial\Lib\site-packages\wradlib\vpr.py:103: DeprecationWarning: ``gridshape`` is not used in <class 'wradlib.vpr.CAPPI'>. It will be removed in wradlib version 2.0.
  warnings.warn(


...

appended image Weather_3D\20230115\RAD_NL62_VOL_NA_202301152345.h5


c:\Users\Aleja\anaconda3\envs\geospatial\Lib\site-packages\wradlib\vpr.py:103: DeprecationWarning: ``gridshape`` is not used in <class 'wradlib.vpr.CAPPI'>. It will be removed in wradlib version 2.0.
  warnings.warn(


appended image Weather_3D\20230115\RAD_NL62_VOL_NA_202301152350.h5


c:\Users\Aleja\anaconda3\envs\geospatial\Lib\site-packages\wradlib\vpr.py:103: DeprecationWarning: ``gridshape`` is not used in <class 'wradlib.vpr.CAPPI'>. It will be removed in wradlib version 2.0.
  warnings.warn(


appended image Weather_3D\20230115\RAD_NL62_VOL_NA_202301152355.h5


In [3]:
import pickle
import numpy as np
import os

dataset = {}

X_train = []
Y_train = []
X_val = []
Y_val = []
X_test = []
Y_test = []

for file in os.listdir("Sequences"):

    # Load the numpy array from the file
    imgs = np.load(f"Sequences/{file}")
    it = 0

    # Split the data based on the year in the file name
    if file[:4] in ["2020", "2021", "2022", "2023"]:
        # Split the data into training sequences and corresponding target sequences
        for i in range((len(imgs)//12)-1):
            X_train.append(imgs[it:it+12])
            Y_train.append(imgs[it+12:it+24])
            it += 12
    elif file[:4] == "2019":
        # Split the data into validation sequences and corresponding target sequences
        for i in range((len(imgs)//24)-1):
            X_val.append(imgs[it:it+12])
            Y_val.append(imgs[it+12:it+24])
            it += 24
    elif file[:4] == "2018":
        # Split the data into testing sequences and corresponding target sequences
        for i in range((len(imgs)//24)-1):
            X_test.append(imgs[it:it+12])
            Y_test.append(imgs[it+12:it+24])
            it += 24
    
dataset["X_train"] = np.array(X_train)
dataset["Y_train"] = np.array(Y_train)
dataset["X_val"] = np.array(X_val)
dataset["Y_val"] = np.array(Y_val)
dataset["X_test"] = np.array(X_test)
dataset["Y_test"] = np.array(Y_test)

# save as a pkl file
with open('dataset.pkl', 'wb') as f:
    pickle.dump(dataset, f)